# Transfehr von C1 zu C2-C5

In [1]:
from lxml import etree
import locale

# Setze die Locale auf Deutsch, um korrekte Sortierung von Umlauten zu gewährleisten
locale.setlocale(locale.LC_ALL, 'de_DE.utf8')

# Pfad zur Input-Datei (vmf_c1.xsd)
input_xsd_file = 'vmf_c1.xsd'

# Pfade zu den Output-Dateien (vmf_c2.xsd, vmf_c3.xsd, vmf_c4.xsd, vmf_c5.xsd)
output_xsd_files = ['vmf_c1.xsd','vmf_c2.xsd', 'vmf_c3.xsd', 'vmf_c4.xsd', 'vmf_c5.xsd']

# Namespace definieren
ns = {'xs': 'http://www.w3.org/2001/XMLSchema'}

# xs:schema-Datei (vmf_c1.xsd) einlesen und alle xs:enumeration-Elemente extrahieren
def extract_enumeration_elements(xsd_file):
    parser = etree.XMLParser(remove_comments=False)
    tree = etree.parse(xsd_file, parser)
    root = tree.getroot()
    
    # Alle xs:enumeration-Elemente finden
    enumerations = root.findall('.//xs:enumeration', namespaces=ns)
    
    # Liste der gesamten xs:enumeration-Elemente mit allen untergeordneten Knoten
    enumeration_elements = [(enum, enum.get('value')) for enum in enumerations if enum.get('value') is not None]
    
    # Alphanumerische Sortierung basierend auf dem value-Attribut unter Berücksichtigung der deutschen Locale
    enumeration_elements.sort(key=lambda x: locale.strxfrm(x[1]))
    
    # Nur die Elemente zurückgeben, nicht die Werte
    return [enum[0] for enum in enumeration_elements]

# xs:enumeration-Elemente in den Output-Dateien ersetzen
def replace_enumeration_elements(output_xsd_file, new_enumerations):
    parser = etree.XMLParser(remove_comments=False)
    tree = etree.parse(output_xsd_file, parser)
    root = tree.getroot()
    
    # Einen geeigneten Elternknoten für xs:enumeration finden
    restriction = root.find('.//xs:restriction', namespaces=ns)
    
    if restriction is None:
        print(f"Kein <xs:restriction>-Element in {output_xsd_file} gefunden. Überspringe Datei.")
        return
    
    # Entfernen der alten xs:enumeration-Elemente
    for enum in restriction.findall('.//xs:enumeration', namespaces=ns):
        restriction.remove(enum)
    
    # Hinzufügen der neuen xs:enumeration-Elemente (bereits sortiert)
    for new_enum in new_enumerations:
        restriction.append(new_enum)  # Die neuen kompletten Elemente anhängen
    
    # Die veränderte Datei speichern
    tree.write(output_xsd_file, pretty_print=True, xml_declaration=True, encoding='UTF-8')

# Hauptfunktion
def main():
    # Schritt 1: Alle xs:enumeration-Elemente aus vmf_c1.xsd extrahieren (inklusive Unterelemente)
    new_enumerations = extract_enumeration_elements(input_xsd_file)
    
    # Schritt 2: Die Werte in den Output-Dateien ersetzen
    for output_file in output_xsd_files:
        replace_enumeration_elements(output_file, new_enumerations)
        print(f"Die xs:enumeration-Elemente in {output_file} wurden ersetzt.")

if __name__ == "__main__":
    main()

Die xs:enumeration-Elemente in vmf_c1.xsd wurden ersetzt.
Die xs:enumeration-Elemente in vmf_c2.xsd wurden ersetzt.
Die xs:enumeration-Elemente in vmf_c3.xsd wurden ersetzt.
Die xs:enumeration-Elemente in vmf_c4.xsd wurden ersetzt.
Die xs:enumeration-Elemente in vmf_c5.xsd wurden ersetzt.
